In [1]:
import pandas as pd
import sqlite3
import plotly.express as px
from tqdm import tqdm
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
conn = sqlite3.connect("data/database.db")

In [2]:
df = pd.read_sql_query("SELECT * FROM price_data", conn)

In [3]:
df.head()

,date,ticker,close,high,low,open,volume,garman_klass_vol,rsi,bb_low,...,return_2d,return_3d,return_6d,return_9d,return_12d,Mkt-RF,SMB,HML,RMW,CMA
0,2015-01-27 00:00:00,A,38.750000,39.250000,38.580002,38.700001,1703500.0,-0.002277,42.551154,3.572676,...,-0.000773,-0.007624,0.002167,-0.002268,-0.004470,-0.013229,-0.040100,0.049200,0.050537,-0.007434
1,2015-01-27 00:00:00,AAPL,27.285000,28.120001,27.257500,28.105000,382274800.0,-0.007667,45.914745,3.195314,...,-0.017141,-0.009763,0.004893,-0.001093,-0.002072,0.073535,-0.020393,-0.015092,0.155138,0.208851
2,2015-01-27 00:00:00,ABBV,63.099998,63.500000,62.200001,62.330002,6532300.0,-0.059758,41.454726,3.738412,...,0.005352,0.000635,-0.003754,-0.000509,-0.005141,0.072632,0.045400,-0.073537,0.175087,-0.034093
3,2015-01-27 00:00:00,ABT,43.680000,44.020000,43.430000,43.880001,4847400.0,-0.013836,39.157346,3.615259,...,-0.002054,-0.006106,-0.003058,-0.002338,-0.003285,-0.030629,0.048403,0.014877,0.151120,-0.068622
4,2015-01-27 00:00:00,ACGL,19.953333,20.160000,19.683332,19.760000,1379700.0,-0.000351,58.501939,2.967072,...,0.003780,0.001060,0.001709,0.001308,0.000321,-0.064490,-0.056542,0.171240,-0.082922,-0.165375
